In [1]:
import pandas as pd

# Load the dataset
data = pd.read_csv('MMM_test_data.csv')

In [2]:
# Convert 'start_of_week' to datetime format with error handling
data['start_of_week'] = pd.to_datetime(data['start_of_week'], errors='coerce')

# Check for any rows where conversion failed
invalid_dates = data[data['start_of_week'].isnull()]
if not invalid_dates.empty:
    print(f"Warning: Found {len(invalid_dates)} rows with invalid dates.")

# Drop rows with invalid dates (if applicable)
data = data.dropna(subset=['start_of_week'])

# Sort data by 'start_of_week' (if not already sorted)
data = data.sort_values('start_of_week').reset_index(drop=True)

# Separate features (spend channels) and target (revenue)
features = data[['spend_channel_1', 'spend_channel_2', 'spend_channel_3',
                 'spend_channel_4', 'spend_channel_5', 'spend_channel_6', 'spend_channel_7']]
target = data['revenue']


print(data['start_of_week'].head())
print(data.head())

0   2020-01-11
1   2020-04-10
2   2020-06-09
3   2020-06-12
4   2020-08-11
Name: start_of_week, dtype: datetime64[ns]
  start_of_week    revenue  spend_channel_1  spend_channel_2  spend_channel_3  \
0    2020-01-11  213117.17           370.19            98.93         24509.24   
1    2020-04-10  195581.04          3655.19           525.06         18024.45   
2    2020-06-09  186425.68          2634.01           108.66          8760.28   
3    2020-06-12  150423.19          1372.79           141.30         10183.94   
4    2020-08-11  202321.02          1723.65           136.25         35550.52   

   spend_channel_4  spend_channel_5  spend_channel_6  spend_channel_7  
0          4982.32          8945.98          7837.48         17768.76  
1          9739.47         20804.05         25445.63         30394.41  
2          4560.60         12747.70         12338.18         22473.45  
3          7226.42         14808.32         14997.14         20745.24  
4          4847.53         14683.10

/tmp/ipykernel_59427/4232500959.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['start_of_week'] = pd.to_datetime(data['start_of_week'], errors='coerce')


In [6]:
from aesara import config
import pymc as pm
import pandas as pd

# Ensure Aesara configurations are set
config.on_opt_error = 'raise'
config.mode = 'FAST_RUN'
config.blas.ldflags = '-L/home/faiq/miniconda3/envs/mmm_env/lib -lblas'

# Load and preprocess data
data = pd.read_csv('MMM_test_data.csv')
data['start_of_week'] = pd.to_datetime(data['start_of_week'], format='%d-%m-%y')
data = data.sort_values('start_of_week').reset_index(drop=True)
features = data[['spend_channel_1', 'spend_channel_2', 'spend_channel_3',
                 'spend_channel_4', 'spend_channel_5', 'spend_channel_6', 'spend_channel_7']]
target = data['revenue']

# Simplified model with reduced channels for beta
with pm.Model() as model:
    alpha = pm.Normal('alpha', mu=0, sigma=100)
    beta = pm.Normal('beta', mu=0, sigma=100, shape=2)  # Reduce shape for testing
    sigma = pm.HalfCauchy('sigma', beta=10)

    adstock_decay = pm.Uniform('adstock_decay', lower=0, upper=1)
    adstock_lag = pm.DiscreteUniform('adstock_lag', lower=1, upper=12)

    adstocked_spend = pm.Deterministic('adstocked_spend', pm.math.dot(features, adstock_decay ** adstock_lag))
    mu = alpha + pm.math.dot(adstocked_spend, beta)

    likelihood = pm.Normal('revenue', mu=mu, sigma=sigma, observed=target)

# Perform sampling
with model:
    trace = pm.sample(1000, tune=1000, cores=1)

# Display posterior summary
print(pm.summary(trace).round(2))


ERROR (aesara.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (aesara.graph.rewriting.basic): node: InplaceDimShuffle{}(TensorConstant{(1,) of 2})
ERROR (aesara.graph.rewriting.basic): TRACEBACK:
ERROR (aesara.graph.rewriting.basic): Traceback (most recent call last):
  File "/home/faiq/miniconda3/envs/mmm_env/lib/python3.8/site-packages/aesara/graph/rewriting/basic.py", line 1933, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/home/faiq/miniconda3/envs/mmm_env/lib/python3.8/site-packages/aesara/graph/rewriting/basic.py", line 1092, in transform
    return self.fn(fgraph, node)
  File "/home/faiq/miniconda3/envs/mmm_env/lib/python3.8/site-packages/aesara/tensor/rewriting/basic.py", line 1141, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
  File "/home/faiq/miniconda3/envs/mmm_env/lib/python3.8/site-packages/aesara/link/c/op.py", line 131, in make_thunk
    return self.make_c_


You can find the C code in this temporary file: /tmp/aesara_compilation_error_wkd03ovz


CompileError: Compilation failed (return status=1):
/home/faiq/miniconda3/envs/mmm_env/bin/g++ -shared -g -O3 -fno-math-errno -Wno-unused-label -Wno-unused-variable -Wno-write-strings -Wno-c++11-narrowing -fno-exceptions -fno-unwind-tables -fno-asynchronous-unwind-tables -march=skylake -mmmx -mno-3dnow -msse -msse2 -msse3 -mssse3 -mno-sse4a -mcx16 -msahf -mmovbe -maes -mno-sha -mpclmul -mpopcnt -mabm -mno-lwp -mfma -mno-fma4 -mno-xop -mbmi -msgx -mbmi2 -mno-pconfig -mno-wbnoinvd -mno-tbm -mavx -mavx2 -msse4.2 -msse4.1 -mlzcnt -mno-rtm -mno-hle -mrdrnd -mf16c -mfsgsbase -mrdseed -mprfchw -madx -mfxsr -mxsave -mxsaveopt -mno-avx512f -mno-avx512er -mno-avx512cd -mno-avx512pf -mno-prefetchwt1 -mclflushopt -mxsavec -mxsaves -mno-avx512dq -mno-avx512bw -mno-avx512vl -mno-avx512ifma -mno-avx512vbmi -mno-avx5124fmaps -mno-avx5124vnniw -mno-clwb -mno-mwaitx -mno-clzero -mno-pku -mno-rdpid -mno-gfni -mno-shstk -mno-avx512vbmi2 -mno-avx512vnni -mno-vaes -mno-vpclmulqdq -mno-avx512bitalg -mno-avx5

In [ ]:
# Example of ROI calculation (adjust as per your model structure)
posterior_means = pm.summary(trace)['mean']
spend_means = posterior_means[['beta__{}'.format(i) for i in range(7)]]
roi_per_channel = (target.mean() / spend_means).sort_values(ascending=False)

print("ROI per channel:")
print(roi_per_channel)


In [ ]:
from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Bayesian Mixed-Media Model Report', 0, 1, 'C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')

    def chapter_body(self, body):
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, body)

# Create PDF document
pdf = PDF()
pdf.add_page()
pdf.chapter_title('Model Results')
pdf.chapter_body(pm.summary(trace).round(2).to_string())
pdf.output('MMM_Report.pdf')